Tentativa de data augmentation

In [ ]:
import pandas as pd
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/rdc/tei

Mounted at /gdrive
/gdrive/MyDrive/rdc/tei


In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in train_df.columns:
    if col != 'id':
        train_df[col] = le.fit_transform(train_df[col])

for col in test_df.columns:
    if col != 'id':
        test_df[col] = le.fit_transform(test_df[col])
    if col == 'class':
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        print(le_name_mapping)

In [ ]:
X = train_df.drop(['id', 'class'], axis=1)
y = train_df['class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Não funcionou

In [ ]:

smote = SMOTE(sampling_strategy='auto', k_neighbors=10, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
import xgboost as xgb

best_params = {
    'max_depth': 9,
    'subsample': 0.9276592101046933,
    'colsample_bytree': 0.9119456082162385,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'random_state': 42
}

model = xgb.XGBClassifier(**best_params)
model.fit(X_resampled, y_resampled)

In [ ]:
test_predictions = model.predict(test_df.drop('id', axis=1))

test_df['class'] = test_predictions
test_df['class'].replace({0: 'e', 1: 'p'}, inplace=True)
test_df[["id","class"]].to_csv("aumento.csv", index=False)